In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import datetime
import sys
import json

In [2]:
features_json_data = open('./features.json').read()
features = json.loads(features_json_data)

In [3]:
from data_extraction import *

DB user name: maxpoon
DB address: localhost
DB port number: 5432


In [38]:
stays = getICUStayPatients()

KeyboardInterrupt: 

In [ ]:
stays.head(10)

In [ ]:
stays['intime'][0]

In [ ]:
getPatientTimeSeries(stays.iloc[0])

In [ ]:
getPatientTimeSeries(stays.iloc[11575])

In [4]:
allTimeSeries = fillNaForAllPatients(None)

In [ ]:
allTimeSeries[1].columns

In [ ]:
allTimeSeriesConcat = pd.concat(allTimeSeries)

In [ ]:
feature = 'Heart Rate'
minValue = allTimeSeriesConcat[feature].min()
maxValue = allTimeSeriesConcat[feature].max()
diff = maxValue - minValue
allTimeSeriesConcat[feature] = (allTimeSeriesConcat[feature]-minValue)/diff

In [ ]:
allTimeSeriesConcat['Heart Rate']

In [ ]:
allTimeSeries[0].columns

In [5]:
features = ['age', 'gender', 'Heart Rate',
       'Respiratory Rate', 'SpO2/SaO2', 'pH', 'Potassium', 'Calcium',
       'Glucose', 'Sodium', 'HCO3', 'White Blood Cells', 'Hemoglobin',
       'Red Blood Cells', 'Platelet Count', 'Weight', 'Urea Nitrogen',
       'Creatinine', 'Blood Pressure', '6 hours urine output', 'gcs',
       'ventilation', 'vasoactive medications', 'sedative medications',
       'Next 6 hours urine output', 'AKI']

In [6]:
allTimeSeries = [timeSeries[features] for timeSeries in allTimeSeries]

In [7]:
len(allTimeSeries[20].columns)

26

In [ ]:
allTimeSeriesConcat = pd.concat(allTimeSeries)
ratio = len(allTimeSeriesConcat)/allTimeSeriesConcat['AKI'].sum()
ratio

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
train, test = train_test_split(allTimeSeries, test_size=0.2, random_state=42)

In [10]:
from data_iterator import *

In [ ]:
iterator = DataIterator(allTimeSeries)

In [ ]:
x, y, sequenceLengths, musks = iterator.next_batch(64)

In [ ]:
from bidirectional_rnn import *

In [ ]:
graph = buildGraph()

In [11]:
import tensorflow as tf

/usr/local/Cellar/python3/3.6.4_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [ ]:
trainGraph(graph, sess, train, test)

In [12]:
import multi_layer_lstm

In [34]:
tf.reset_default_graph()

In [35]:
graph = multi_layer_lstm.buildGraph(stateSizes=(512, 256))

In [36]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [37]:
multi_layer_lstm.trainGraph(graph, sess, train, test)

Accuracy after epoch 1  - tr: 0.74880454972 - te: 0.773959241692
Accuracy after epoch 2  - tr: 0.75702324195 - te: 0.772211079003
Accuracy after epoch 3  - tr: 0.769675080231 - te: 0.797000288538
Accuracy after epoch 4  - tr: 0.766849216278 - te: 0.791004943757
Accuracy after epoch 5  - tr: 0.771334313033 - te: 0.801499653157
Accuracy after epoch 6  - tr: 0.786515414744 - te: 0.789249404726
Accuracy after epoch 7  - tr: 0.769775461413 - te: 0.799219964896
Accuracy after epoch 8  - tr: 0.779772189966 - te: 0.791962933819
Accuracy after epoch 9  - tr: 0.786261612593 - te: 0.807174459622
Accuracy after epoch 10  - tr: 0.78008008135 - te: 0.78871101209


In [ ]:
feed = {graph['x']: x, graph['y']: y, graph['seqlen']: sequenceLengths, graph['musk']: musks}

In [ ]:
sess.run(graph['predicts'], feed_dict=feed)

In [ ]:
import multi_layer_bidirectional_rnn

In [ ]:
tf.reset_default_graph()

In [ ]:
multi_graph = multi_layer_bidirectional_rnn.buildGraph()

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [ ]:
multi_layer_bidirectional_rnn.trainGraph(multi_graph, sess, train, test)

In [ ]:
multi_layer_bidirectional_rnn.trainGraph(multi_graph, sess, train, test, 60)